# Data reading example 2 - PRIMAP-hist v2.2 #
To run this example the file *PRIMAPHIST22__19-Jan-2021.csv* must be placed in the same folder as this notebook.
The PRIMAP-hist data (doi:10.5281/zenodo.4479172) is available from Zenodo: https://zenodo.org/record/4479172

In [ ]:
# imports
import primap2 as pm2

## Dataset Specifications ##
Here we define which columns of the csv file contain the metadata. The dict `coords_cols` contains the mapping of csv columns to PRIMAP2 metadata.
Default values are set using `coords_defaults`. The terminologies (e.g. IPCC2006 for categories or the ISO3 country codes for area) are set in the coords_terminologies dict. `meta_mapping` defines conversion of metadata values, e.g. category codes. You can either specify a dict for a metadata column which directly defines the mapping or a string which governs which function to use to create the mapping. Possible functions are hard-coded currently for security reasons. `filter_keep` and `filter_remove` filter the input data. Each entry in `filter_keep` specifies a subset of the input data which is kept while the subsets defined by `filter_remove` are removed from the input data.

For details we refer to the documentation of `read_wide_csv_file_if` which is located in the `io` module of PRIMAP2.

In [9]:
file = "PRIMAPHIST22__19-Jan-2021.csv"
folder = "."
coords_cols = {
    "unit": "unit",
    "entity": "entity",
    "area": "country",
    "scenario": "scenario",
    "category": "category",
}
coords_defaults = {
    "source": "PRIMAP-hist_v2.2",
    "citation": "DOI: 10.5281/zenodo.4479172",
}
coords_terminologies = {
    "area": "ISO3",
    "category": "IPCC2006",
    "scenario": "PRIMAP-hist",
}

meta_mapping = {
    "category": "PRIMAP1",
    "unit": "PRIMAP1",
    "entity": "PRIMAP1",
}

filter_keep = {
    "f1": {
        "entity": "CO2",
        "category": ["IPC2", "IPC1"],
        "country": ["AUS", "BRA", "CHN", "GBR", "AFG"],
    },
    "f2": {
        "entity": "KYOTOGHG",
        "category": ["IPCMAG", "IPC4"],
        "country": ["AUS", "BRA", "CHN", "GBR", "AFG"],
    },
}

filter_remove = {"f1": {"scenario": "HISTTP"}}
# filter_keep = {"f1": {"entity": "KYOTOGHG", "category": ["IPC2", "IPC1"]},}
# filter_keep = {}
# filter_remove = {}

## Reading the data to interchange format ##
To enable a wider use of the PRIMAP2 data reading functionality we read into the PRIMAP2 interchange format which is a wide format pandas DataFrame with metadata in columns and following PRIMAP2 specifications. The attributes (`attrs`) are stored in `DataFrame.attrs`. As the `attrs` functionality in pandas is experiemental it is just stored in the DataFrame returned by the reading functions and should be stored individually before doing any processing with the DataFrame.

Here we read the data using the `read_wide_csv_file_if()` function. We have specified restrictive filters above to limit the data included in this notebook.

In [9]:
PMH_if = pm2.io.read_wide_csv_file_if(
    file,
    folder,
    coords_cols=coords_cols,
    coords_defaults=coords_defaults,
    coords_terminologies=coords_terminologies,
    meta_mapping=meta_mapping,
    filter_keep=filter_keep,
    filter_remove=filter_remove,
)
PMH_if.head()

,scenario (PRIMAP-hist),area (ISO3),category (IPCC2006),entity,unit,1850,1851,1852,1853,1854,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,source
3123,HISTCR,AFG,1,CO2,Gg CO2 / yr,0.147,0.155,0.163,0.172,0.182,...,8440.0,12200.0,10700.0,9990.0,11000.0,11700.0,12700.0,13100.0,18600.0,PRIMAP-hist_v2.2
3136,HISTCR,AUS,1,CO2,Gg CO2 / yr,0.000,0.000,0.000,0.000,0.000,...,380000.0,378000.0,383000.0,376000.0,372000.0,380000.0,389000.0,393000.0,393000.0,PRIMAP-hist_v2.2
3152,HISTCR,BRA,1,CO2,Gg CO2 / yr,92.100,96.900,102.000,108.000,114.000,...,349000.0,363000.0,395000.0,424000.0,445000.0,431000.0,401000.0,403000.0,389000.0,PRIMAP-hist_v2.2
3161,HISTCR,CHN,1,CO2,Gg CO2 / yr,11.000,11.600,12.200,12.900,13.600,...,7730000.0,8550000.0,8810000.0,9010000.0,9090000.0,9070000.0,9030000.0,9190000.0,9400000.0,PRIMAP-hist_v2.2
3193,HISTCR,GBR,1,CO2,Gg CO2 / yr,123000.000,117000.000,117000.000,116000.000,140000.000,...,495000.0,454000.0,472000.0,460000.0,421000.0,406000.0,384000.0,373000.0,366000.0,PRIMAP-hist_v2.2


## Transformation to PRIMAP2 xarray format ##
The transformation to PRIMAP2 xarray format is done using the function `from_interchange_format` which takes an interchange format DataFrame, the `attrs` dict and an optional parameter specifying the data format as inputs. The resulting xr Dataset is already quantified, thus the variables are pint arrays which include a unit.

In [10]:
PMH_pm2 = pm2.io.from_interchange_format(PMH_if, PMH_if.attrs)
PMH_pm2

Magnitude,[[[[[0.147] [0.169] [nan] [nan]] [[0.0] [0.0] [nan] [nan]] [[92.1] [78.4] [nan] [nan]] [[11.0] [0.0] [nan] [nan]] [[123000.0] [605.0] [nan] [nan]]]] [[[[0.155] [0.178] [nan] [nan]] [[0.0] [0.0] [nan] [nan]] [[96.9] [82.6] [nan] [nan]] [[11.6] [0.0] [nan] [nan]] [[117000.0] [575.0] [nan] [nan]]]] [[[[0.163] [0.188] [nan] [nan]] [[0.0] [0.0] [nan] [nan]] [[102.0] [87.0] [nan] [nan]] [[12.2] [0.0] [nan] [nan]] [[117000.0] [573.0] [nan] [nan]]]] ... [[[[12700.0] [293.0] [nan] [nan]] [[389000.0] [19100.0] [nan] [nan]] [[401000.0] [78500.0] [nan] [nan]] [[9030000.0] [1330000.0] [nan] [nan]] [[384000.0] [14000.0] [nan] [nan]]]] [[[[13100.0] [292.0] [nan] [nan]] [[393000.0] [19400.0] [nan] [nan]] [[403000.0] [79500.0] [nan] [nan]] [[9190000.0] [1340000.0] [nan] [nan]] [[373000.0] [14400.0] [nan] [nan]]]] [[[[18600.0] [298.0] [nan] [nan]] [[393000.0] [19800.0] [nan] [nan]] [[389000.0] [79300.0] [nan] [nan]] [[9400000.0] [1380000.0] [nan] [nan]] [[366000.0] [13600.0] [nan] [nan]]]]]
Units,CO2 gigagram/a
Magnitude,[[[[[nan] [nan] [155.0] [615.0]] [[nan] [nan] [911.0] [17300.0]] [[nan] [nan] [165.0] [1090.0]] [[nan] [nan] [15000.0] [18700.0]] [[nan] [nan] [11400.0] [8170.0]]]] [[[[nan] [nan] [154.0] [668.0]] [[nan] [nan] [922.0] [17300.0]] [[nan] [nan] [168.0] [1140.0]] [[nan] [nan] [15000.0] [20900.0]] [[nan] [nan] [11500.0] [8190.0]]]] [[[[nan] [nan] [154.0] [719.0]] [[nan] [nan] [934.0] [17400.0]] [[nan] [nan] [170.0] [1190.0]] [[nan] [nan] [14900.0] [23000.0]] [[nan] [nan] [11700.0] [8230.0]]]] ... [[[[nan] [nan] [3800.0] [13900.0]] [[nan] [nan] [10700.0] [64700.0]] [[nan] [nan] [63900.0] [450000.0]] [[nan] [nan] [182000.0] [848000.0]] [[nan] [nan] [17500.0] [37700.0]]]] [[[[nan] [nan] [3900.0] [13800.0]] [[nan] [nan] [10800.0] [68300.0]] [[nan] [nan] [65500.0] [456000.0]] [[nan] [nan] [187000.0] [830000.0]] [[nan] [nan] [17800.0] [38000.0]]]] [[[[nan] [nan] [4010.0] [13300.0]] [[nan] [nan] [10800.0] [66800.0]] [[nan] [nan] [67100.0] [444000.0]] [[nan] [nan] [192000.0] [820000.0]] [[nan] [nan] [18000.0] [37600.0]]]]]
Units,CO2 gigagram/a
